# Recurrent neural network (RNN) from scratch

# What is a recurrent neural network?

A recurrent neural network (RNN), as opposed to a regular fully connected neural network (FCNN), has layers that are connected to themselves.

The difference might be clearer by first looking at an FCNN.

<img src="Figures/fcnn.svg" width="40%" alt="FCNN">

In an FCNN there are no connections between nodes in a single layer. For instance, $h_1^1$ is not connected to $h_2^1$. In addition, the input and output are always of a fixed length.

In an RNN, however, this is no longer the case. Nodes in the hidden layers are connected to themselves, represented by the curved lines in the figure below.

<img src="Figures/rnn.svg" width="40%">

Thus the output $\vec{h}$ from the hidden layer is fed back into the hidden layer. This recurrence makes RNNs useful when working with sequential data, as we can have input of variable length. This is more clear if we unfold the recurrent part of the network.

<img src="Figures/rnn_unfold.svg" width="40%">

# The mathematics of RNNs

## The RNN architecture

Consider some sequential input $X$ with $n$ features. Note that $X$ here is an array with two axes, since it contains $n$ features at each time step in the sequence. We will denote the input at a specific time step $t$ as
$$\vec{X}^{(t)} = \begin{pmatrix}
X^{(t)}_1 \\ \vdots \\ X^{(t)}_n
\end{pmatrix},$$
which is then an $n$-dimensional vector.

Next, consider an RNN with $L$ hidden layers, and an output layer with $m$ features. We will denote the output of the $l$'th hidden layer at time step $t$ as
$$\vec{h}_l^{(t)} = \begin{pmatrix}
h_{l, 1}^{(t)} \\ \vdots \\ h_{l, n_l}^{(t)}
\end{pmatrix},$$
with $n_l$ being the number of features in the $l$'th hidden layer. The output of the RNN at time step $t$ is denoted
$$\hat{\vec{y}}^{(t)} = \begin{pmatrix}
\hat{y}_1 \\ \vdots \\ \hat{y}_m,
\end{pmatrix}$$
where the hat is there to distinguish the RNN output $\hat{\vec{y}}^{(t)}$ from the target value, which is denoted $\vec{y}^{(t)}$.
The RNN will then look like this.

<img src="Figures/large_rnn.svg" width="50%">

## Forward propagation

Now let $W_x$ and $\vec{b}_x$ be the weight matrix and bias vector relating the input to the first hidden layer, and let $\sigma_1$ be the activation function in the first hidden layer. The output from the first hidden layer at the first time step is then
$$ \vec{h}_1^{(1)} = \sigma_1 \left(W_x \vec{X}^{(1)} + \vec{b}_x \right). $$
To simplify our notation, we define
$$ \vec{z}_1^{(1)} = W_x \vec{X}^{(1)} + \vec{b}_x, $$
such that
$$ \vec{h}_1^{(1)} = \sigma_1 (\vec{z}_1^{(1)}). $$

At later time steps than the first, we will also need to consider the contribution from the previous time step. If we define $W_h^{11}$ and $\vec{b}_h^{11}$ to be the weight matrix and bias vector relating the hidden node at time step $t-1$ to the node at time step $t$, both in the first hidden layer, we get the following equations for propagating forward through the first hidden layer.
\begin{align}
\vec{z}_1^{(t)} &= W_x \vec{X}^{(t)} + \vec{b}_x + W_h^{11} \vec{h}_1^{(t-1)} + \vec{b}_h^{11}
\\
\vec{h}_1^{(t)} &= \sigma_1 (\vec{z}_1^{(t)}).
\end{align}

You might see a pattern in the notation here. Let $W_h^{l_1 l_2}$ and $\vec{b}_h^{l_1 l_2}$ be the weight matrix and bias vector relating nodes in the $l_1$'th layer to the $l_2$'th layer. If $l_1 = l_2-1$ we are looking at a connection between two nodes at the same time step, but in subsequent layers; and if $l_1 = l_2$ we are looking at a connection between two nodes in the same layer, but at subsequent time steps. Let $\sigma_l$ be the activation function for nodes in the $l$'th hidden layer. We then have the following equations for forward propagation through the node.
\begin{align}
\vec{z}_l^{(t)} &= W_h^{l-1,l} \vec{h}_{l-1}^{(t)} + \vec{b}_h^{l-1,l} + W_h^{ll} \vec{h}_l^{t-1} + \vec{b}_h^{ll}
\\
\vec{h}_l^{(t)} &= \sigma_l (\vec{z}_l^{(t)}).
\end{align}

This expression is valid up to $l=L$, which is the last hidden layer. When feeding forward to the output layer we will no longer consider previous time steps. Let $W_y$ and $\vec{b}_y$ denote the weight matrix and bias vector of the output layer, and let $\sigma_y$ be the output activation function. We then get the following equations for forward propagation through the output layer.
\begin{align}
\vec{o}^{(t)} &= W_y \vec{h}_L^{(t)} + \vec{b}_y
\\
\hat{\vec{y}}^{(t)} &= \sigma_y (\vec{o}^{(t)}).
\end{align}

---

To simplify notation we will rename 
$$
W_x \to W^{01} \\
\vec{b}_x \to \vec{b}^{01} \\
W_y \to W^{L,L+1} \\
\vec{b}_y \to \vec{b}^{L,L+1} \\
W_h^{l_1 l_2} \to W^{l_1 l_2} \\
\vec{b}_h^{l_1 l_2} \to \vec{b}^{l_1 l_2}
$$

---

In order to propagate forward through the network we need some weights and biases to connect the nodes. To simplify the notation going forward, we will consider the input layer to be the *zeroth layer*, and the output layer to be the *$L+1$'th layer*. We need each node to propagate to the node at the next layer (keeping the time step constant), and the next time step (keeping the layer constant), except for the input and output layers which do not connect to each other (as illustrated in the diagram above).

Let $W_{l,l+1}$ be the weight matrix and $\vec{b}_{l,l+1}$ the bias vector, both connecting nodes at the $l$'th layer to the $l+1$'th layer, keeping the time step constant. Next, let $W_{ll}$ be the weight matrix and $\vec{b}_{ll}$ the bias vector, both connecting nodes at subsequent time steps in the same layer. Also, let $\sigma_l$ be the activation function in the $l$'th layer. Lastly, define the weighted sum $\vec{z}_l^{(t)}$ at layer $l$ and time step $t$ such that the output of the node is the activation of that weighted sum, that is, such that $\vec{h}_l^{(t)} = \sigma_l (\vec{z}_l^{(t)})$.

Using these definitions the output from the first hidden layer at the first time step is then
$$ \vec{h}_1^{(1)} = \sigma_1 \left( \vec{z}_1^{(1)} \right), $$
with
$$ \vec{z}_1^{(1)} = W_{01} \vec{X}^{(1)} + \vec{b}_{01}.$$

At later time steps we will also need to consider the contribution from the previous time step. Hence for $t \geq 2$ we will define
$$\left( \vec{z}_1^{(t)} \right)_\text{layer} = W_{01} X^{(t)} + \vec{b}_{01}$$
$$\left( \vec{z}_1^{(t)} \right)_\text{time} = W_{11} \vec{h}_1^{(t-1)} + \vec{b}_{11},$$
such that $\left( \vec{z}_1^{(t)} \right)_\text{layer}$ is the contribution from the previous layer, and $\left( \vec{z}_1^{(t)} \right)_\text{time}$ is the contribution from the previous time step. We then have
$$\vec{z}_1^{(t)} = \left( \vec{z}_1^{(t)} \right)_\text{layer} + \left( \vec{z}_1^{(t)} \right)_\text{time},$$
and
$$\vec{h}_1^{(t)} = \sigma_1 \left( \vec{z}_1^{(t)} \right).$$

The expression is exactly the same for any hidden node, but for $l \geq 2$ we substitute $\vec{X}^{(t)}$ with $\vec{h}_{l-1}^{(t)}$. Thus for the $l$'th layer and $t$'th time step we have
$$ \left( \vec{z}_l^{(t)} \right)_{layer} = W_{l-1,l} \vec{h}_{l-1}^{(t)} + \vec{b}_{l-1,l} $$
and
$$ \left( \vec{z}_l^{(t)} \right)_{time} = W_{ll} \vec{h}_{l}^{(t-1)} + \vec{b}_{ll}, $$
that combine to give
$$ \vec{z}_l^{(t)} = \left( \vec{z}_l^{(t)} \right)_{layer} + \left( \vec{z}_l^{(t)} \right)_{time}, $$
which in turn results in
$$ \vec{h}_l^{(t)} = \sigma_l \left( \vec{z}_l^{(t)} \right). $$
This is also valid at the first time step by setting $\left( \vec{z}_l^{(1)} \right)_\text{time} = 0$.

The expression for the output layer is exactly the same as above, but with $\left( \vec{z}_l^{(t)} \right)_\text{time} = 0$. Thus we have
$$ \vec{z}_{L+1}^{(t)} = \left( \vec{z}_{L+1}^{(t)} \right)_\text{layer} = W_{L,L+1} \vec{h}_L^{(t)} + \vec{b}_{L,L+1} $$
and
$$ \hat{\vec{y}}^{(t)} = \sigma_{L+1} \left( \vec{z}_{L+1}^{(t)} \right) $$

The equations given for the forward propagation can seem a bit messy, so it is nice to have a more visual aid of what is going on. Here is a diagram of the complete RNN including the weights and biases relating the different nodes.

<img src="Figures/feed_forward.svg" width="50%">

And here is a weights and biases connected to a single arbitrary node. The green arrows represent input to the node, and the red arrows represent the output from the node.

<img src="Figures/feed_forward_node.svg" width=30%>

## Backpropagation through time (BPTT)

Backpropagation in an RNN works by comparing the output of the network to some target output (just as in the regular neural network), and propagating backwards through both the layers and the *time sequence*. It is therefore commonly referred to as *backpropagation through time* (BPTT). We will now derive the necessary equations to perform BPTT.

We assume that we have propagated forward through the network, and have produced some output $\hat{\vec{y}}^{(t)}$. We want to compare this with some target output value $\vec{y}^{(t)}$, and will do so through a cost function $C (\hat{\vec{y}}, \vec{y})$. Note that, although the cost *function* $C$ is the same at all time steps, the *value* it gives will be different at different time steps. We will denote the cost at time step $t$ by $C^{(t)} \left(\hat{\vec{y}}^{(t)}, \vec{y}^{(t)} \right)$.

From the cost function at each time step, we want to compute the gradient with respect to each weight and bias, that is, we want to compute
$$
\frac{\partial C^{(t)}}{\partial W_x} \; , \; \frac{\partial C^{(t)}}{\partial \vec{b}_x} \\
\frac{\partial C^{(t)}}{\partial W_y} \; , \; \frac{\partial C^{(t)}}{\partial \vec{b}_y} \\
\frac{\partial C^{(t)}}{\partial W_h^{ij}} \; , \; \frac{\partial C^{(t)}}{\partial \vec{b}_h^{ij}}
$$